In [2]:
import brickschema
from brickschema.namespaces import BRICK, RDFS, OWL, TAG, RDF, SH
from rdflib import URIRef
import networkx

from utils import *
from rules.relationship_relaxation import *
from rules.type_relaxation import *
from relaxation_graphs.fast_relaxation_graph import *
from evaluation.metrics_fast import *

# Not ideal visual just validation
import matplotlib.pyplot as plt
import networkx as nx
import datetime
import os
import pickle

import pymortar
from queries.mortar import queries as q1
from queries.cbe import queries as q2
from queries.nrel import queries as q3

brick_graph = brickschema.Graph(load_brick=True)
query_bank = q1 + q2 + q3

In [3]:
files = os.listdir('Data - Brick models/')
brick_models = []

with open("brick_models.pkl", 'rb') as fp:
    brick_models = pickle.load(fp)
    

In [3]:
# for file in files:
#     if file.endswith(".ttl"):
#         g = brickschema.Graph(load_brick=True)
#         g.load_file('Data - Brick models/'+file)
#         g.expand(profile="owlrl")
#         brick_models.append(g)

# with open("brick_models.pkl", 'wb') as fp:
#     pickle.dump(brick_models, fp)


In [4]:
max_level = 5
query_metrics = {}
for query in q1:
    print(query)
    start_time = datetime.datetime.now()
    G = get_fast_relaxed_graph(query=query, max_level=max_level)
    end_time = datetime.datetime.now()
    query_metrics[query]={'graph': G}
    query_metrics[query]['relaxation_time']  = (end_time-start_time).total_seconds()
    query_metrics[query]['query_size'] = len(extract_triples(query))
    print(query_metrics[query])

SELECT * WHERE { ?s ?p ?o }
{'graph': <networkx.classes.graph.Graph object at 0x7ff1f8f29190>, 'relaxation_time': 0.000653, 'query_size': 1}
SELECT ?point ?point_type WHERE {
    ?point rdf:type brick:Point .
    ?point rdf:type ?point_type 
}
{'graph': <networkx.classes.graph.Graph object at 0x7ff2094e2a00>, 'relaxation_time': 0.000161, 'query_size': 2}
SELECT ?meter WHERE { ?meter rdf:type brick:Green_Button_Meter }
{'graph': <networkx.classes.graph.Graph object at 0x7ff248180070>, 'relaxation_time': 0.000393, 'query_size': 1}
 SELECT ?t WHERE { ?t rdf:type brick:Weather_Temperature_Sensor }
{'graph': <networkx.classes.graph.Graph object at 0x7ff2094e2820>, 'relaxation_time': 0.000144, 'query_size': 1}
SELECT ?sensor WHERE {
    ?sensor rdf:type brick:Zone_Air_Temperature_Sensor .
    ?sensor brick:isPointOf ?equip 
}
{'graph': <networkx.classes.graph.Graph object at 0x7ff2094e2ac0>, 'relaxation_time': 0.000752, 'query_size': 2}
SELECT ?sp WHERE {
    ?sp rdf:type brick:Zone_Air_Temp

In [5]:
buildings = brick_models[:5]

In [ ]:
for query in q1:
    relaxed_graph = query_metrics[query]['graph']
    select_statement = query.split("{")[0] + "{\n"
    
    building_nodes_with_data = {}
    for building_idx in range(len(buildings)):
        building_model = buildings[building_idx]
        nodes_with_data= []
        
        for node_idx, node in relaxed_graph.nodes().data():
            brick_query = generate_brick_query_from_node(node['query'], select_statement=select_statement)
            res = run_brick_query(building_model=building_model, query=brick_query)

            if len(res) > 0:
                nodes_with_data.append(node)
        building_nodes_with_data[building_idx] = len(nodes_with_data)
    query_metrics[query]['building_nodes_with_data'] = building_nodes_with_data
    query_metrics[query]['size_relaxed_graph'] = len(relaxed_graph.nodes())
    print(query_metrics[query])

{'graph': <networkx.classes.graph.Graph object at 0x7ff1f8f29190>, 'relaxation_time': 0.000653, 'query_size': 1, 'building_nodes_with_data': {0: 9, 1: 13, 2: 13, 3: 13, 4: 13}, 'size_relaxed_graph': 17}
{'graph': <networkx.classes.graph.Graph object at 0x7ff2094e2a00>, 'relaxation_time': 0.000161, 'query_size': 2, 'building_nodes_with_data': {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}, 'size_relaxed_graph': 1}
{'graph': <networkx.classes.graph.Graph object at 0x7ff248180070>, 'relaxation_time': 0.000393, 'query_size': 1, 'building_nodes_with_data': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}, 'size_relaxed_graph': 2}
{'graph': <networkx.classes.graph.Graph object at 0x7ff2094e2820>, 'relaxation_time': 0.000144, 'query_size': 1, 'building_nodes_with_data': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}, 'size_relaxed_graph': 2}
{'graph': <networkx.classes.graph.Graph object at 0x7ff2094e2ac0>, 'relaxation_time': 0.000752, 'query_size': 2, 'building_nodes_with_data': {0: 12, 1: 12, 2: 12, 3: 12, 4: 12}, 'size_relaxed_graph': 12}

In [ ]:
# file = "Data - Brick models/artx.ttl"

q1 = """SELECT ?damper ?vav WHERE {
    ?damper   rdf:type   brick:Damper . 
    ?vav    rdf:type   brick:VAV . 
    ?damper   brick:isPartOf  ?vav   
}"""

# q2 = """SELECT ?damper ?vav WHERE {
#     ?damper   rdf:type   brick:Damper .
#     ?vav      rdf:type   brick:VAV .
#     ?damper   brick:isPartOf  ?vav  
# }"""

In [ ]:
# q1_res = get_fast_evaluation_metrics(file="Data - Brick models/artx.ttl", query=q1)
# q1_res

In [12]:
c = pymortar.Client("https://beta-api.mortardata.org")
print(q1)
sites = c.qualify([q1]).sites

SELECT ?damper ?vav WHERE {
    ?damper   rdf:type   brick:Damper . 
    ?vav    rdf:type   brick:VAV . 
    ?damper   brick:isPartOf  ?vav   
}


In [16]:
df = c.qualify([q1])

In [ ]:
brickschema.